## 가상환경
- BERT, SBERT, SimCSE: s-bert
- DiffCSE: diffcse
- PromCSE: promcse

In [1]:
import torch
import numpy as np
import csv
from tqdm.notebook import tqdm

from covid_q.code.get_align_uniform import *

In [2]:
align_csv = 'covid_q/data/positive_pairs.csv'
uniform_csv = 'covid_q/data/train4_uniform.csv'
uniform_all_csv = 'covid_q/data/final_master_dataset.csv'

## Output Embedding Vector Normalized 여부 확인

In [3]:
from numpy import linalg as LA

test_sentence = 'This framework generates embeddings for each input sentence'

In [4]:
# 가상환경 s-bert
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
from simcse import SimCSE

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')
model_sbert = SentenceTransformer('nli-roberta-base')
model_simcse = SimCSE('princeton-nlp/unsup-simcse-bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
12/05/2022 19:52:57 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: nli-roberta-base
12/05/2022 19:53:02 - INFO

In [5]:
# 가상환경 s-bert
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
from simcse import SimCSE

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')
model_sbert = SentenceTransformer('nli-roberta-base')
model_simcse = SimCSE('princeton-nlp/unsup-simcse-bert-base-uncased')

print('BERT:', LA.norm(get_embedding(test_sentence, tokenizer_bert, model_bert), 2)) # normalize하는 코드 추가함
print('SBERT:', LA.norm(model_sbert.encode(test_sentence), 2))
print('SimCSE:', LA.norm(model_simcse.encode(test_sentence), 2))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
12/05/2022 19:53:08 - INFO - sentence_transformers.SentenceTransformer -   Load pretrained SentenceTransformer: nli-roberta-base
12/05/2022 19:53:09 - INFO

BERT: 0.9999997


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

SBERT: 21.112795


100%|██████████| 1/1 [00:00<00:00, 77.82it/s]

SimCSE: 1.0


In [4]:
# 가상환경 diffcse
from DiffCSE.diffcse import DiffCSE

model_diffcse = DiffCSE('voidism/diffcse-bert-base-uncased-sts')

print('DiffCSE:', LA.norm(model_diffcse.encode(test_sentence), 2))

Some weights of BertModel were not initialized from the model checkpoint at voidism/diffcse-bert-base-uncased-sts and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/05/2022 20:03:48 - INFO - DiffCSE.diffcse.tool -   Use `cls_before_pooler` for DiffCSE models. If you want to use other pooling policy, specify `pooler` argument.
100%|██████████| 1/1 [00:00<00:00, 30.19it/s]

DiffCSE: 1.0000001


In [ ]:
# 가상환경 promcse

# promcse_encoder.py로 확인한 결과:
# PromCSE: 1.0

## BERT

In [3]:
from transformers import BertModel, BertTokenizer

In [4]:
# BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_add = BertTokenizer.from_pretrained('bert-base-uncased')
model_add = BertModel.from_pretrained('bert-base-uncased')

# Add vocabulary
new_tokens = ['covid']
num_added_toks = tokenizer_add.add_tokens(new_tokens)
model_add.resize_token_embeddings(len(tokenizer_add))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predic

Embedding(30523, 768)

### Alignment

In [5]:
%%time
# Vocab 추가
q1_emb_add, q2_emb_add = get_all_embeddings_align(align_csv, tokenizer_add, model_add)
print('Alignment (Vocab added):', align_loss(q1_emb_add, q2_emb_add).numpy())

# Vocab 추가 x
q1_emb, q2_emb = get_all_embeddings_align(align_csv, tokenizer, model)
print('Alignment:', align_loss(q1_emb, q2_emb).numpy())

Alignment (Vocab added): 0.5184952
Alignment: 0.46573868
CPU times: user 35min 24s, sys: 1min 31s, total: 36min 55s
Wall time: 1min 37s


### Uniformity

In [6]:
%%time
# train4_uniform.csv 사용

# Vocab 추가
q_emb_add = get_all_embeddings_uniform(uniform_csv, tokenizer_add, model_add)
print('Uniformity (Vocab added):', uniform_loss(q_emb_add).numpy())

# Vocab 추가 x
q_emb = get_all_embeddings_uniform(uniform_csv, tokenizer, model)
print('Uniformity:', uniform_loss(q_emb).numpy())

Uniformity (Vocab added): -1.4912271
Uniformity: -1.3666482
CPU times: user 12min 2s, sys: 30.9 s, total: 12min 33s
Wall time: 33 s


In [7]:
%%time
# final_master_dataset.csv 사용

# Vocab 추가
q_emb_add = get_all_embeddings_uniform_all(uniform_all_csv, tokenizer_add, model_add)
print('Uniformity (All) (Vocab added):', uniform_loss(q_emb_add).numpy())

# Vocab 추가 x
q_emb = get_all_embeddings_uniform_all(uniform_all_csv, tokenizer, model)
print('Uniformity (All):', uniform_loss(q_emb).numpy())

Uniformity (All) (Vocab added): -1.4989414
Uniformity (All): -1.386892
CPU times: user 55min 30s, sys: 2min 13s, total: 57min 43s
Wall time: 2min 31s


## SBERT

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
%%time

model_list = [   
    'nli-bert-base',
    'nli-roberta-base',
    'stsb-bert-base',
    'stsb-roberta-base',
    'bert-base-nli-stsb-mean-tokens',
    'roberta-base-nli-stsb-mean-tokens'
    ]

test_sentence = 'This framework generates embeddings for each input sentence'

for model_name in model_list:
    
    model = SentenceTransformer(model_name)
    
    # Embedding dimension 체크
    if model.encode(test_sentence).shape[0] != 768:
        print(f'Embedding dimension for {model_name} is not 768')
        break
        
    # Alignment
    q1_emb, q2_emb = get_sentence_embedding_align(align_csv, model, unit=True)
    print(f'[{model_name}] Alignment:', align_loss(q1_emb, q2_emb).numpy())
    
    # Uniformity
    q_emb = get_sentence_embedding_uniform(uniform_csv, model, unit=True) # train4_uniform.csv 사용
    print(f'[{model_name}] Uniformity:', uniform_loss(q_emb).numpy())

    q_emb = get_sentence_embedding_uniform_all(uniform_all_csv, model, unit=True) # final_master_dataset.csv 사용
    print(f'[{model_name}] Uniformity (All):', uniform_loss(q_emb).numpy())

[nli-bert-base] Alignment: 1.1255863
[nli-bert-base] Uniformity: -4.8261204
[nli-bert-base] Uniformity (All): -1.2439708
[nli-roberta-base] Alignment: 1.1763544
[nli-roberta-base] Uniformity: -4.8763666
[nli-roberta-base] Uniformity (All): -1.2442255
[stsb-bert-base] Alignment: 1.2876233
[stsb-bert-base] Uniformity: -5.3902197
[stsb-bert-base] Uniformity (All): -1.3850875
[stsb-roberta-base] Alignment: 1.2763783
[stsb-roberta-base] Uniformity: -5.2100806
[stsb-roberta-base] Uniformity (All): -1.3764973
[bert-base-nli-stsb-mean-tokens] Alignment: 1.2876233
[bert-base-nli-stsb-mean-tokens] Uniformity: -5.3902197
[bert-base-nli-stsb-mean-tokens] Uniformity (All): -1.3850875
[roberta-base-nli-stsb-mean-tokens] Alignment: 1.2763783
[roberta-base-nli-stsb-mean-tokens] Uniformity: -5.2100806
[roberta-base-nli-stsb-mean-tokens] Uniformity (All): -1.3764973
CPU times: user 1min 18s, sys: 14.7 s, total: 1min 33s
Wall time: 26.7 s


## SimCSE

In [10]:
from simcse import SimCSE

In [8]:
%%time

model_list = [   
    'princeton-nlp/unsup-simcse-bert-base-uncased',
    'princeton-nlp/unsup-simcse-roberta-base',
    'princeton-nlp/sup-simcse-bert-base-uncased',
    'princeton-nlp/sup-simcse-roberta-base'
    ]

test_sentence = 'This framework generates embeddings for each input sentence'

for model_name in model_list:
    
    model = SimCSE(model_name)
    
    # Embedding dimension 체크
    if model.encode(test_sentence).shape[0] != 768:
        print(f'Embedding dimension for {model_name} is not 768')
        break
        
    # Alignment
    q1_emb, q2_emb = get_sentence_embedding_align(align_csv, model)
    print(f'[{model_name}] Alignment:', align_loss(q1_emb, q2_emb).numpy())
    
    # Uniformity
    q_emb = get_sentence_embedding_uniform(uniform_csv, model) # train4_uniform.csv 사용
    print(f'[{model_name}] Uniformity:', uniform_loss(q_emb).numpy())

    q_emb = get_sentence_embedding_uniform_all(uniform_all_csv, model) # final_master_dataset.csv 사용
    print(f'[{model_name}] Uniformity (All):', uniform_loss(q_emb).numpy())

12/04/2022 21:38:49 - INFO - simcse.tool -   Use `cls_before_pooler` for unsupervised models. If you want to use other pooling policy, specify `pooler` argument.
100%|██████████| 9/9 [00:00<00:00, 28.33it/s]


[princeton-nlp/unsup-simcse-bert-base-uncased] Alignment: 0.66342306


100%|██████████| 6/6 [00:00<00:00, 28.16it/s]


[princeton-nlp/unsup-simcse-bert-base-uncased] Uniformity: -2.160287


100%|██████████| 26/26 [00:00<00:00, 26.47it/s]


...

[princeton-nlp/sup-simcse-roberta-base] Alignment: 0.7651381


100%|██████████| 6/6 [00:00<00:00, 28.54it/s]


[princeton-nlp/sup-simcse-roberta-base] Uniformity: -2.6867628


100%|██████████| 26/26 [00:00<00:00, 26.54it/s]

[princeton-nlp/sup-simcse-roberta-base] Uniformity (All): -2.7603693
CPU times: user 38.4 s, sys: 5.51 s, total: 43.9 s
Wall time: 19.6 s


## DiffCSE

In [3]:
from DiffCSE.diffcse import DiffCSE

In [4]:
%%time

model_list = [   
    'voidism/diffcse-bert-base-uncased-sts',
    'voidism/diffcse-bert-base-uncased-trans',
    'voidism/diffcse-roberta-base-sts',
    'voidism/diffcse-roberta-base-trans'
    ]

test_sentence = 'This framework generates embeddings for each input sentence'

for model_name in model_list:
    
    model = DiffCSE(model_name)
    
    # Embedding dimension 체크
    if model.encode(test_sentence).shape[0] != 768:
        print(f'Embedding dimension for {model_name} is not 768')
        break
        
    # Alignment
    q1_emb, q2_emb = get_sentence_embedding_align(align_csv, model)
    print(f'[{model_name}] Alignment:', align_loss(q1_emb, q2_emb).numpy())
    
    # Uniformity
    q_emb = get_sentence_embedding_uniform(uniform_csv, model) # train4_uniform.csv 사용
    print(f'[{model_name}] Uniformity:', uniform_loss(q_emb).numpy())

    q_emb = get_sentence_embedding_uniform_all(uniform_all_csv, model) # final_master_dataset.csv 사용
    print(f'[{model_name}] Uniformity (All):', uniform_loss(q_emb).numpy())

Some weights of BertModel were not initialized from the model checkpoint at voidism/diffcse-bert-base-uncased-sts and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/04/2022 21:53:50 - INFO - DiffCSE.diffcse.tool -   Use `cls_before_pooler` for DiffCSE models. If you want to use other pooling policy, specify `pooler` argument.
 33%|███▎      | 2/6 [00:00<00:00, 19.07it/s]

[voidism/diffcse-bert-base-uncased-sts] Alignment: 0.28357688


 12%|█▏        | 3/26 [00:00<00:00, 24.87it/s]

[voidism/diffcse-bert-base-uncased-sts] Uniformity: -0.95707244


100%|██████████| 26/26 [00:00<00:00, 26.74it/s]


...

[voidism/diffcse-roberta-base-trans] Alignment: 0.1858467


 12%|█▏        | 3/26 [00:00<00:00, 23.85it/s]

[voidism/diffcse-roberta-base-trans] Uniformity: -0.6210463


100%|██████████| 26/26 [00:00<00:00, 26.75it/s]

[voidism/diffcse-roberta-base-trans] Uniformity (All): -0.6481909
CPU times: user 48 s, sys: 14.6 s, total: 1min 2s
Wall time: 50.9 s


## PromCSE
- PromCSE는 코드 구조가 달라서 (args 사용함) 바로 python script로 실행
```
cd PromCSE
python promcse_encoder.py
```

- 결과
 - Alignment: 0.90093094
 - Uniformity: -2.7525558
 - Uniformity (All): -2.8021905